In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

# List all the files in the specified directory.
images = os.listdir("/content/drive/MyDrive/Graduation_project/Code/images")

# Set an environmental variable "TORCH_HOME" for PyTorch to use as its root directory.
os.environ["TORCH_HOME"] = "model_weights_edir"

# Load a pre-trained ResNet-18 model from torchvision.
model = torchvision.models.resnet18(weights = "DEFAULT")

# Initialize empty lists to store image names and feature vectors.
names = []
vectors = None

# Set the model to evaluation mode.
model.eval()

# Set the root directory for images.
root = "/content/drive/MyDrive/Graduation_project/Code/images/"


transform = transforms.Compose([
    # Resize the image to 256x256 pixels.
    transforms.Resize((256 , 256)) ,
    # Convert the image to a PyTorch tensor.
    transforms.ToTensor() ,
    # Normalize the image.
    transforms.Normalize(mean = [0.485 , 0.456 , 0.406] , std = [0.229 , 0.224 , 0.225])
])


activation = {}

def get_activation(name):
    def hook(model , input , output):
        activation[name] = output.detach()
    return hook


model.avgpool.register_forward_hook(get_activation("avgpool"))

# Iterate through the images to extract features.
with torch.no_grad():
    for i , file in enumerate(images):
        try:
            img = Image.open(root + file)
            img = transform(img)
            out = model(img[None , ...])
            vec = activation["avgpool"].numpy().squeeze()[None , ...]

            if vectors is None:
                vectors = vec
            else:
                vectors = np.vstack([vectors , vec])
            names.append(file)
        except:
            continue

        if i % 100 == 0 and i != 0:
          # Print progress updates.
            print(i , "done")

# Save the feature vectors and corresponding image names.
np.save("Vectors.npy" , vectors)
np.save("Names.npy" , names)


FileNotFoundError: ignored

In [ ]:
import numpy as np

# Load the .npy file
loaded_data = np.load('Vectors.npy')

# Access the data
print(loaded_data)

**Streamlit (Web Page)**

In [ ]:
! pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from PIL import Image
import time
from scipy.spatial.distance import cdist

st.set_page_config(
    page_title="Project",
    page_icon=":mag:",
)

# Add a custom style block
st.markdown(
    """
    <style>
        .stApp {
            background-color: #F0F0F0; /* Change the background color here */
            color:#22668D;
        }

        .css-1uc4frf {
            text-align: center !important; /* Center align the title */
        }

        .stTitle {
            font-size: 2em !important; /* Adjust the font size of the title */
            margin-bottom: 0.5em !important; /* Adjust the margin to your preference */
        }
    </style>
    """,
    unsafe_allow_html=True,
)

# Add your logo
logo = st.image("https://coc.qu.edu.sa/laravel-filemanager/photos/shares/Logos/COC.png", use_column_width=False, width=200)

# Add your title
st.markdown("<h1 style='text-align: center;color: #22668D;font-family:serif;'>Image Matching Engine</h1>", unsafe_allow_html=True)
@st.cache_data
def read_data():
    vectors = np.load("Vectors.npy")
    names = np.load("Names.npy")
    return vectors , names

vecs , name = read_data()

_ , fcol2 , _ = st.columns(3)

scol1 , scol2 = st.columns(2)

ch = scol1.button("Start / change")
fs = scol2.button("find similar")

if ch:
    random_name = name[np.random.randint(len(name))]
    fcol2.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + random_name))
    st.session_state["disp_img"] = random_name
    st.write(st.session_state["disp_img"])
if fs:
    c1 , c2 , c3 , c4 , c5 = st.columns(5)
    idx = int(np.argwhere(name == st.session_state["disp_img"]))
    target_vec = vecs[idx]
    fcol2.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + st.session_state["disp_img"]))
    top5 = cdist(target_vec[None , ...] , vecs).squeeze().argsort()[1:6]
    c1.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + name[top5[0]]).resize((300, 300)))  # Resize the output images
    c2.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + name[top5[1]]).resize((300, 300)))
    c3.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + name[top5[2]]).resize((300, 300)))
    c4.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + name[top5[3]]).resize((300, 300)))
    c5.image(Image.open("/content/drive/MyDrive/Graduation_project/Code/images/" + name[top5[4]]).resize((300, 300)))



**Five steps to run the Streamlit code.**

In [ ]:
# Install the pyngrok package which allows access to ngrok
!pip install pyngrok

In [ ]:

# Download ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip


In [ ]:
# Unzip ngrok
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
# Add your ngrok authentication token
!ngrok config add-authtoken 2VUTNFyOdyVP6SH8UDWpna9TBXP_4PJ3e1BBPJWfwCGpWAqvD

In [ ]:
from pyngrok import ngrok
# Start the Streamlit app in the background
!streamlit run app.py&>/dev/null&

# Connect ngrok to port 8501 (where your Streamlit app is running)
public_url = ngrok.connect(8501)

# Print the public URL where your app is accessible
public_url